In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
import torch
import cv2
from transformers import CLIPProcessor, CLIPModel

In [3]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

C:\Users\AMD\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AMD\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [30]:
# Load Mean Opinion Scores from CSV File
df = pd.read_csv('All_Stimuli_MOS_and_MLE.csv')
df['Stimuli'] = df['Stimuli'].str.replace('Chameleon', 'Cameleon')

In [ ]:
from PIL import Image

# Load the model and preprocessing
model, preprocess = load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")

# In your loop:
for group_name, image_paths in grouped_images.items():
    print(f"Group {group_name}:")
    entropy_list = list()
    for path in image_paths:
        print(f"  {path}")
        extracted_name = os.path.splitext(os.path.basename(path))[0]

        # MOS Score
        matching_rows = df[df['Stimuli'].str.startswith(extracted_name)]
        mos_r = matching_rows[matching_rows['Stimuli'].str.endswith('_R')]['MOS'].values[0]
        mos_z = matching_rows[matching_rows['Stimuli'].str.endswith('_Z')]['MOS'].values[0]
        avg_mos = (mos_r + mos_z) / 2

        # Entropy Score
        image = imread(path, as_gray=False)
        entropy = shannon_entropy(image)
        entropy_list.append(entropy)

        # CLIP Score
        # Convert to PIL Image for CLIP preprocessing
        pil_image = Image.open(path)
        
        # Preprocess the image
        image = preprocess(pil_image).unsqueeze(0).to(model.device)

        # Get text input
        text_from_row = matching_rows[matching_rows['Stimuli'].str.endswith('_R')]['Source_model'].values[0]
        text_input = clip.tokenize([text_from_row]).to(model.device)

        # Get image and text features
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text_input)

            # Normalize features
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            # Calculate similarity
            similarity_score = (image_features @ text_features.T).squeeze().item()
            print(f"CLIP score: {similarity_score}")

ModuleNotFoundError: No module named 'clip'